In [19]:
import open3d as o3d
import numpy as np
import pandas
from timeit import default_timer as time
import model_gen_functions
from data_extraction import dataExtract
from reconstruct_extract import reconstruct_extract
import tylmen_api_functions

filepath = './Point Clouds/set 1/brandon slow.ply'
pcd = o3d.io.read_point_cloud(filepath)

In [25]:
def cloud_denoise(cloud: o3d.geometry.PointCloud, passes = 5, silent=False):
    #if you know of a more readable way to do this please tell me
    if(silent==False):
        start = time()
        print("Beginning Denoise...")
    
    if(passes <= 0):
        #will always output to console, unsure how you would end up with this happening (command will not accept 0) but here it is
        print("Denoise started with zero passes, aborting")
    if(passes >= 1):
        
        if(silent==False):
        
            print("Initial Count " + str((np.asarray(cloud.points).size)/3) + " Points")
            print("Removing Bad/Dupe points before cleaning")
        prepared_cloud = cloud.remove_duplicated_points()
        prepared_cloud = prepared_cloud.remove_non_finite_points()
        if(silent==False):
            print("New Count " + str((np.asarray(prepared_cloud.points).size)/3) + " Points, beginning passes")
        denoise_cloud, ind = cloud.remove_statistical_outlier(nb_neighbors=15, std_ratio=1.2) #std_ratio is the standard deviation a point must be 
        if(silent==False):
            p1 = time()
            print("Pass 1: " + str((np.asarray(denoise_cloud.points).size)/3) + " Points in " + str(p1 - start) + " seconds")
    if(passes >= 2):
        denoise_cloud, ind = denoise_cloud.remove_statistical_outlier(nb_neighbors=25, std_ratio=1.7)
        if(silent==False):
            p2 = time()
            print("Pass 2: " + str((np.asarray(denoise_cloud.points).size)/3) + " Points in " + str(p2 - p1) + " seconds")
    if(passes >= 3):
        denoise_cloud, ind = denoise_cloud.remove_statistical_outlier(nb_neighbors=100, std_ratio=2.0)
        if(silent==False):
            p3 = time()
            print("Pass 3: " + str((np.asarray(denoise_cloud.points).size)/3) + " Points in " + str(p3 - p2) + " seconds")
    if(passes >= 4):
        denoise_cloud, ind = denoise_cloud.remove_statistical_outlier(nb_neighbors=30, std_ratio=1.5)
        if(silent==False):
            p4 = time()
            print("Pass 4: " + str((np.asarray(denoise_cloud.points).size)/3) + " Points in " + str(p4 - p3) + " seconds")
    if(passes >= 5):
        denoise_cloud, ind = denoise_cloud.remove_statistical_outlier(nb_neighbors=50, std_ratio=1.8)
        if(silent==False):
            p5 = time()
            print("Pass 5: " + str((np.asarray(denoise_cloud.points).size)/3) + " Points in " + str(p5 - p4) + " seconds")

    if(passes > 5):
        if(silent==False):
            pprevious = p5
        for x in range(passes - 5):
            denoise_cloud, ind = denoise_cloud.remove_statistical_outlier(nb_neighbors=30, std_ratio=1.8) #repeatable medium filter
            #going past 7 passes is very likely to completely decimate the model, 5 was already pushing it in some test cases. adjust to needs
            if(silent==False):
                ploop = time()
                print("Pass " + (x+5) +": " + str((np.asarray(denoise_cloud.points).size)/3) + " Points in " + str(ploop - pprevious) + " seconds")
                pprevious = ploop

    if(passes > 0):
        if(silent==False):
            end = time()
            print("Denoise complete in " + str(end - start) + " seconds")
        return denoise_cloud
    else:
        return cloud


In [21]:
cut_pcd = model_gen_functions.cut_floor(pcd)
type(cut_pcd)

Cropping floor
Floor cropped in 1.2353479000012157 seconds


open3d.cpu.pybind.geometry.PointCloud

In [26]:
denoise_cloud = cloud_denoise(cut_pcd, passes=5)

Beginning Denoise...
Initial Count 147298.0 Points
Removing Bad/Dupe points before cleaning
New Count 147298.0 Points, beginning passes
Pass 1: 139307.0 Points in 0.41462890000002517 seconds
Pass 2: 137475.0 Points in 0.3809578000000329 seconds
Pass 3: 135659.0 Points in 1.2069179999998596 seconds
Pass 4: 127430.0 Points in 0.49048559999937424 seconds
Pass 5: 127384.0 Points in 0.4501801999995223 seconds
Denoise complete in 2.943255099999078 seconds


In [27]:
o3d.visualization.draw_geometries([denoise_cloud])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [8]:
def cloud_denoise2(cloud, stat_passes = 3, hush = False):
    #basic steps i should have added ages ago
    prepared_cloud = cloud.remove_duplicated_points()
    prepared_cloud = prepared_cloud.remove_non_finite_points()

    #bounds = prepared_cloud.get_axis_aligned_bounding_box() #Didn't expect me, did ya?
    #high_bound = bounds.get_max_bound()[2] 
    #low_bound = bounds.get_min_bound()[2]
    #height = high_bound - low_bound #necessary for radius steps, as to prevent errors, the radius will be a percentage of the height
    
    #The Washing Machine
    denoised_cloud, ind = prepared_cloud.remove_statistical_outlier(nb_neighbors=25, std_ratio=1.2)
    denoised_cloud, ind = denoised_cloud.remove_statistical_outlier(nb_neighbors=100, std_ratio=1.8)
    denoised_cloud, ind = denoised_cloud.remove_statistical_outlier(nb_neighbors=40, std_ratio=1.8)
    #denoised_cloud, ind = denoised_cloud.remove_radius_outlier(nb_neighbors=40, radius=float(0.08 * height))
    
    return denoised_cloud
    

In [9]:
denoise2_cloud = cloud_denoise2(cut_pcd)

TypeError: remove_radius_outlier(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cpu.pybind.geometry.PointCloud, nb_points: int, radius: float, print_progress: bool = False) -> Tuple[open3d.cpu.pybind.geometry.PointCloud, List[int]]

Invoked with: PointCloud with 134926 points.; kwargs: nb_neighbors=30, radius=0.03001359388232231

In [ ]:
o3d.visualization.draw_geometries([denoise_cloud])